In [2]:
import pandas as pd
from ast import literal_eval
import matplotlib.pyplot as plt
plt.style.use('seaborn')

In [3]:
namekey = pd.read_csv('cleanData/cleanIMDB/imdb_namekey.csv', index_col=0) # import name key

In [4]:
comp = pd.read_csv('cleanData/imdb_combined_prof.csv') # import movie composite dataset - info and ROI % info

In [5]:
# remove unnecessary columns
comp.drop(axis=1, labels=['Unnamed: 0', 'Unnamed: 0_x', 'Unnamed: 0_y', 'id', 'concat'], inplace=True)

In [6]:
comp.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,directors,writers,...,ROI_percent,ROI_usd,year,month,inflation,worldwide_gross_inf,domestic_gross_inf,production_budget_inf,ROI_USD_inf,ROI%_inf
0,tt1137470,Accidental Love,Accidental Love,2015,100.0,"['Comedy', 'Romance']",4.1,7048.0,['nm0751102'],"['nm0330751', 'nm0799052', 'nm1001124']",...,-0.994791,-25864564,2015,March,1.15,155751.40,0.00,29900000.0,-29744248.60,-99.479092
1,tt1403241,Wolves,Wolves,2014,91.0,"['Action', 'Fantasy', 'Horror']",5.3,7312.0,['nm0371684'],['nm0371684'],...,-0.994725,-17905047,2014,November,1.15,109195.95,0.00,20700000.0,-20590804.05,-99.472483
2,tt1125929,Love Ranch,Love Ranch,2010,117.0,"['Biography', 'Comedy', 'Drama']",5.6,2742.0,['nm0000431'],['nm0414893'],...,-0.994154,-24853851,2010,June,1.25,182686.25,172356.25,31250000.0,-31067313.75,-99.415404
3,tt1545106,Vamps,Vamps,2012,92.0,"['Comedy', 'Horror', 'Romance']",5.2,8838.0,['nm0002132'],['nm0002132'],...,-0.994074,-15905188,2012,November,1.19,112826.28,3999.59,19040000.0,-18927173.72,-99.407425
4,tt1486193,5 Days of War,5 Days of War,2011,113.0,"['Action', 'Drama', 'War']",5.6,13089.0,['nm0001317'],"['nm1209140', 'nm3649361']",...,-0.992684,-11912207,2011,August,1.21,106229.53,21149.59,14520000.0,-14413770.47,-99.268392


## Step 1: Prepare dataframe for grouping

In [22]:
# if possible, convert list-looking strings into actual lists for downstream df.explode()
def lit_try(val):
    try:
        return literal_eval(val) # function which parses strings to find type
    except:
        return val # pass null values or other non-strings to avoid exceptions

In [23]:
comp.iloc[:,8:22] = comp.iloc[:,8:22].applymap(lit_try) # convert role column values into lists

In [24]:
# add two director columns together (columns from two different sources)
comp['director'] = comp['directors'] + comp['director_pr']

In [25]:
# add two writer columns together (columns from two different sources)
comp['writer'] = comp['writers'] + comp['writer_pr']

In [26]:
# add actor and actress columns to make gender neutral column
comp['actor/actress'] = comp['actor'] + comp['actress']

### Select most important roles

Of all the available roles, let's select all but the production designer, archive footage, archive sound, and self

In [27]:
# list of roles we want to analyze
pplorder = ['actor','actress','actor/actress','director','writer','producer','cinematographer','editor','composer']